In [ ]:
%load_ext kamu

In [ ]:
%import_dataset us.cityofnewyork.data.ems-incident-dispatch --alias dispatch

In [ ]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW dispatch_serious AS (
    SELECT
        cad_incident_id,
        incident_datetime,
        initial_call_type,
        initial_severity_level_code,
        final_call_type,
        final_severity_level_code,
        first_assignment_datetime,
        valid_dispatch_rspns_time_indc,
        dispatch_response_seconds_qy,
        first_activation_datetime,
        first_on_scene_datetime,
        valid_incident_rspns_time_indc,
        incident_response_seconds_qy,
        incident_travel_tm_seconds_qy,
        first_to_hosp_datetime,
        first_hosp_arrival_datetime,
        incident_close_datetime,
        held_indicator,
        incident_disposition_code,
        borough,
        incident_dispatch_area,
        zipcode,
        policeprecinct,
        citycouncildistrict,
        communitydistrict,
        communityschooldistrict,
        congressionaldistrict,
        reopen_indicator,
        special_event_indicator,
        standby_indicator,
        transfer_indicator
    FROM dispatch
    WHERE standby_indicator = false 
        AND transfer_indicator = false
        AND special_event_indicator = false
)

In [ ]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW dst AS (
    SELECT DATE("2013-03-10") AS date, "start" as dst UNION ALL
    SELECT DATE("2013-11-03") AS date, "end" as dst UNION ALL
    SELECT DATE("2014-03-09") AS date, "start" as dst UNION ALL
    SELECT DATE("2014-11-02") AS date, "end" as dst UNION ALL
    SELECT DATE("2015-03-08") AS date, "start" as dst UNION ALL
    SELECT DATE("2015-11-01") AS date, "end" as dst UNION ALL
    SELECT DATE("2016-03-13") AS date, "start" as dst UNION ALL
    SELECT DATE("2016-11-06") AS date, "end" as dst UNION ALL
    SELECT DATE("2017-03-12") AS date, "start" as dst UNION ALL
    SELECT DATE("2017-11-05") AS date, "end" as dst UNION ALL
    SELECT DATE("2018-03-11") AS date, "start" as dst UNION ALL
    SELECT DATE("2018-11-04") AS date, "end" as dst
)

In [ ]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW groups AS (
    SELECT "week" as set, "before" as offset, -7 as from, 0 as to
    UNION ALL
    SELECT "week" as set, "after" as offset, 0 as from, 7 as to
    UNION ALL
    SELECT "month" as set, "before" as offset, -30 as from, 0 as to
    UNION ALL
    SELECT "month" as set, "after" as offset, 0 as from, 30 as to
    UNION ALL
    SELECT "monday" as set, "before" as offset, -6 as from, -5 as to
    UNION ALL
    SELECT "monday" as set, "after" as offset, 1 as from, 2 as to
    UNION ALL
    SELECT "control_prev_weeks" as set, "before" as offset, -14 as from, -7 as to
    UNION ALL
    SELECT "control_prev_weeks" as set, "after" as offset, -7 as from, 0 as to
    UNION ALL
    SELECT "control_next_weeks" as set, "before" as offset, 7 as from, 14 as to
    UNION ALL
    SELECT "control_next_weeks" as set, "after" as offset, 14 as from, 21 as to
)

In [ ]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW dst_groups AS (
    SELECT g.set, g.offset, d.dst, date_add(d.date, g.from) as from, date_add(d.date, g.to) as to
    FROM dst as d
    CROSS JOIN groups as g
)

In [ ]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW labeled AS (
    SELECT g.set, g.offset, g.dst, d.*
    FROM dst_groups as g
    INNER JOIN dispatch_serious as d
        ON d.incident_datetime >= g.from AND d.incident_datetime < g.to
)

In [ ]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW grouped AS (
    SELECT set, offset, dst, final_call_type as type, count(*) as count
        FROM labeled
        GROUP BY set, offset, dst, final_call_type
)

In [ ]:
%%sql
CREATE OR REPLACE TEMPORARY VIEW deltas AS (
    SELECT 
        bf.set, bf.dst, bf.type, 
        bf.count as count_before,
        af.count as count_after,
        (bf.count + af.count) / 2 as count_avg,
        (af.count - bf.count) as delta,
        (af.count - bf.count) / ((bf.count + af.count) / 2) * 100 as delta_pct
    FROM (SELECT * FROM grouped WHERE offset = 'before') as bf
    JOIN (SELECT * FROM grouped WHERE offset = 'after') as af
        ON bf.set = af.set AND bf.dst = af.dst AND bf.type = af.type
)

In [ ]:
%%sql -o plt_data
SELECT * FROM deltas

In [ ]:
%%local
import pandas_bokeh
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Select
from bokeh.layouts import row, column
from bokeh.palettes import Category20

pandas_bokeh.output_notebook()

types = list(sorted(plt_data.type.unique().tolist()))
palette = Category20[20]
color_map = {name: palette[i % len(palette)] for i, name in enumerate(types)}

def get_data(set):
    start = plt_data[(plt_data.set == set) & (plt_data.dst == 'start')].to_dict('list')
    end = plt_data[(plt_data.set == set) & (plt_data.dst == 'end')].to_dict('list')
    end['count_avg'] = [-v for v in end['count_avg']]
    for k in start:
        start[k].extend(end[k])
    start['color'] = [color_map[t] for t in start['type']]
    return start

source = ColumnDataSource(data=get_data('control_next_weeks'))

tooltips = [
    ("type", "@type"),
    ("before", "@count_before"),
    ("after", "@count_after"),
    ("delta", "@delta"),
    ("delta pct", "@delta_pct%"),
]

p = figure(tooltips=tooltips)
p.circle(source=source, x='delta', y='count_avg', color='color', fill_alpha=0.5, size=8)

show(p)